In [ ]:
stock_companies = {
    'ADANIPORTS': ['ADANIPORTS', 'Adani Ports'],
    'ASIANPAINT': ['ASIANPAINT', 'Asian Paints'],
    'AXISBANK': ['AXISBANK', 'Axis Bank'],
    'BAJAJ-AUTO': ['BAJAJ-AUTO', 'Bajaj Auto'],
    'BAJAJFINSV': ['BAJAJFINSV', 'Bajaj Finserv'],
    'BAJFINANCE': ['BAJFINANCE', 'Bajaj Finance'],
    'BHARTIARTL': ['BHARTIARTL', 'Bharti Airtel', 'Airtel'],
    'BRITANNIA': ['BRITANNIA', 'Britannia Industries'],
    'BPCL': ['BPCL', 'Bharat Petroleum'],
    'COALINDIA': ['COALINDIA', 'Coal India'],
    'EICHERMOT': ['EICHERMOT', 'Eicher Motors'],
    'DRREDDY': ['DRREDDY', "Dr. Reddy's Laboratories"],
    'GAIL': ['GAIL', 'Gas Authority of India'],
    'GRASIM': ['GRASIM', 'Grasim Industries'],
    'CIPLA': ['CIPLA', 'Cipla Limited'],
    'HCLTECH': ['HCLTECH', 'HCL Technologies'],
    'HDFCBANK': ['HDFCBANK', 'HDFC Bank'],
    'HEROMOTOCO': ['HEROMOTOCO', 'Hero MotoCorp'],
    'HINDUNILVR': ['HINDUNILVR', 'Hindustan Unilever', 'HUL'],
    'HINDALCO': ['HINDALCO', 'Hindalco Industries'],
    'ICICIBANK': ['ICICIBANK', 'ICICI Bank'],
    'INDUSINDBK': ['INDUSINDBK', 'IndusInd Bank'],
    'INFY': ['INFY', 'Infosys'],
    'IOC': ['IOC', 'Indian Oil'],
    'ITC': ['ITC', 'ITC Limited'],
    'JSWSTEEL': ['JSWSTEEL', 'JSW Steel'],
    'KOTAKBANK': ['KOTAKBANK', 'Kotak Mahindra Bank'],
    'LT': ['LT', 'Larsen & Toubro'],
    'MARUTI': ['MARUTI', 'Maruti Suzuki'],
    'MM': ['MM', 'Mahindra & Mahindra'],
    'NESTLEIND': ['NESTLEIND', 'Nestle India'],
    'NTPC': ['NTPC', 'NTPC Limited'],
    'ONGC': ['ONGC', 'Oil and Natural Gas Corporation'],
    'POWERGRID': ['POWERGRID', 'Power Grid Corporation'],
    'RELIANCE': ['RELIANCE', 'Reliance Industries'],
    'SBIN': ['SBIN', 'State Bank of India'],
    'SHREECEM': ['SHREECEM', 'Shree Cement'],
    'SUNPHARMA': ['SUNPHARMA', 'Sun Pharma'],
    'TATAMOTORS': ['TATAMOTORS', 'Tata Motors'],
    'TATASTEEL': ['TATASTEEL', 'Tata Steel'],
    'TCS': ['TCS', 'Tata Consultancy Services'],
    'TECHM': ['TECHM', 'Tech Mahindra'],
    'TITAN': ['TITAN', 'Titan Company'],
    'ULTRACEMCO': ['ULTRACEMCO', 'UltraTech Cement'],
    'UPL': ['UPL', 'UPL Limited'],
    'VEDL': ['VEDL', 'Vedanta'],
    'WIPRO': ['WIPRO', 'Wipro Limited'],
    'ZEEL': ['ZEEL', 'Zee Entertainment']
}
companies = {}

for stock, names in stock_companies.items():
  companies[stock]=[name.lower() for name in names]


In [17]:
import pandas as pd
import json
import os

all_stocks = {}
for filename in os.listdir('/content/drive/MyDrive/stocks_new'):
    if filename.endswith('.csv'):
        try:
            filepath = os.path.join('/content/drive/MyDrive/stocks_new', filename)
            df = pd.read_csv(filepath)
            df = df.dropna(subset=['Close'])
            df = df[df['Close'] != '']

            company_name = filename[:-4]  # Remove the .csv extension

            if 'Close' in df.columns and 'Date' in df.columns:
                # Remove rows where 'Close' is NaN or empty
                df = df.dropna(subset=['Close'])
                df = df[df['Close'] != '']

                # Convert 'Close' column to numeric, handling commas
                df['Close'] = df['Close'].astype(str).str.replace(',', '', regex=False)
                df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
                df = df.dropna(subset=['Close']) # Drop rows with invalid numeric values
                all_stocks[company_name] = {
                  'close': df['Close'].values.tolist(),
                  'date': df['Date'].values.tolist()
                }
        except Exception as e:
            print(f"Error processing file {filename}: {e}")


with open('/content/drive/MyDrive/stocks_new/all_stocks.json', 'w') as f:
    json.dump(all_stocks, f)

In [18]:
import json

all_stocks = {}
with open('/content/drive/MyDrive/stocks_new/all_stocks.json', 'r') as f:
    all_stocks = json.load(f)

articles_similarities = []
with open('/content/drive/MyDrive/articles/similarities.json', 'r') as f:
    articles_similarities = json.load(f)

In [19]:
print(len(all_stocks), len(articles_similarities))

48 34110


In [20]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Function to get the stock prices for a given date range
def get_stock_prices(stock, start_date):
    if stock in all_stocks:
        dates = all_stocks[stock]['date']
        try:
            start_index = dates.index(start_date)
        except ValueError:
            # Find the next immediate date
            start_index = next((i for i, date in enumerate(dates) if date > start_date), None)
            if start_index is None:
                return None  # No date found after the start_date

        end_index = start_index + 14
        if end_index <= len(all_stocks[stock]['close']):
            return all_stocks[stock]['close'][start_index:end_index]
        else:
            return None  # Not enough data points
    else:
        return None  # Stock not found

def get_company_articles(company):
  company_articles = []
  for article in articles_similarities:
    if article['current_company'] == company:
      company_articles.append(article)
  return company_articles

# Function to clean and convert stock prices to floats
def clean_stock_prices(stock_prices):
    cleaned_prices = []
    for price in stock_prices:
        if isinstance(price, str):
            price = price.replace(',', '')  # Remove commas
        cleaned_prices.append(float(price))
    return np.array(cleaned_prices)

# Prepare the data for each stock
def prepare_data(stock, articles, stock_prices, dates, sequence_length=60, influence_length=14):
    sequences = []
    labels = []

    # Convert dates to a pandas Series for easy date manipulation
    dates_series = pd.Series(dates)

    # Clean and convert stock prices to floats
    stock_prices = clean_stock_prices(stock_prices)

    # Initialize scalers
    scaler_stock_prices = StandardScaler()
    scaler_difference_vector = StandardScaler()

    stock_prices_scaled = scaler_stock_prices.fit_transform(stock_prices.reshape(1,-1)).flatten()

    for i in range(len(stock_prices) - (sequence_length + 30)):
        # Historical data
        historical_data = stock_prices_scaled[i:i + sequence_length]

        # Get the current date
        current_date = pd.to_datetime(dates[i + sequence_length])

        # Influenced data
        influenced_data = []
        for j in range(influence_length):
            # Calculate the date for the influenced period
            influenced_date = current_date - pd.Timedelta(days=influence_length - j)

            # Find the article for the influenced date
            article = next((a for a in articles if a['current_date'] == influenced_date), None)

            if article:
                most_similar_index = article['most_similar_index']
                most_similar_company = article['most_similar_company']
                most_similar_date = article['most_similar_date']
                difference_vector = np.array(article['differece_vector'])

                # Get the stock prices for the most similar company
                similar_prices = get_stock_prices(most_similar_company, most_similar_date)
                # Scale the similar prices
                similar_prices_scaled = scaler_stock_prices.transform(similar_prices.reshape(-1, 1)).flatten()

                # Append the scaled difference vector and scaled similar prices separately
                influenced_data.append((difference_vector, similar_prices_scaled))
            else:
                # Handle the case where no article is available for the influenced date
                influenced_data.append((np.zeros(768), np.zeros(14)))

        sequences.append((historical_data, influenced_data))
        labels.append(stock_prices[i + sequence_length:i + sequence_length + 30])

    return sequences, np.array(labels), scaler_stock_prices

# Example usage
stock = 'ADANIPORTS'
stock_prices = all_stocks[stock]['close']
dates = all_stocks[stock]['date']
articles = get_company_articles(stock)
sequences, y_data, scaler_stock_prices = prepare_data(stock, articles, stock_prices, dates)

In [ ]:
print(len(sequences),len(y_data))

4126 4126


In [30]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dropout, Dense, Concatenate, Flatten

from tensorflow.keras import backend as K
K.clear_session()

# Define the shape of the historical data
historical_data_shape = (60, 1)  # Assuming stock_prices is a list of lists or a 2D array

# Main input: 60-day sequence data
main_input = Input(shape=historical_data_shape, name='main_input')

# Process the main input
x = Conv1D(filters=256, kernel_size=3, activation='relu')(main_input)
x = MaxPooling1D(pool_size=2)(x)
x = LSTM(128, return_sequences=True)(x)
x = Dropout(0.2)(x)
x = LSTM(128, return_sequences=True)(x)
x = Dropout(0.2)(x)
x = LSTM(128, return_sequences=True)(x)
x = Dropout(0.2)(x)
x = LSTM(64, return_sequences=False)(x)
x = Dropout(0.1)(x)

# Influenced input: 14-day influenced data
influenced_input = Input(shape=(14, 768 + 14), name='influenced_input')

# Separate the difference vector and similar prices
# difference_vector_input = Input(shape=(768,), name='difference_vector_input')
# similar_prices_input = Input(shape=(14,), name='similar_prices_input')

# Process the influenced input directly
y = Dense(128, activation='relu')(influenced_input)
y = Dropout(0.2)(y)
y = Dense(64, activation='relu')(y)
y = Dropout(0.2)(y)
y = Flatten()(y)

# Flatten the influenced input
# flattened_influenced = Flatten()(influenced_input)

# Merge the outputs
merged = Concatenate()([x, y])

# Continue with further layers
output = Dense(128, activation='relu')(merged)
output = Dropout(0.2)(output)
output = Dense(64, activation='relu')(output)
output = Dropout(0.2)(output)
output = Dense(30)(output)  # Output layer for predicting the next 30 trading days

# Define the model
model = Model(inputs=[main_input, influenced_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [27]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ main_input (InputLayer)   │ (None, 60, 1)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 58, 256)        │          1,024 │ main_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d             │ (None, 29, 256)        │              0 │ conv1d[0][0]           │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 29, 128)        │        197,120 │ max_pooling1d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 29, 128)        │              0 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ (None, 29, 128)        │        131,584 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ influenced_input          │ (None, 14, 782)        │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 29, 128)        │              0 │ lstm_1[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 14, 128)        │        100,224 │ influenced_input[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ (None, 29, 128)        │        131,584 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 14, 128)        │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 29, 128)        │              0 │ lstm_2[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 14, 64)         │          8,256 │ dropout_4[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ (None, 64)             │         49,408 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)       │ (None, 14, 64)         │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 64)             │              0 │ lstm_3[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 896)            │              0 │ dropout_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 960)            │              0 │ dropout_3[0][0],       │
│                           │                        │                │ flatten[0][0]          │
├──────────────────────

 Total params: 752,414 (2.87 MB)

 Trainable params: 752,414 (2.87 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
K.clear_session()


# Prepare the data for training
X_main = np.array([seq[0] for seq in sequences]).reshape(-1, 60, 1)
X_influenced = np.array([np.hstack([np.hstack((dv, sp)) for dv, sp in seq[1]]) for seq in sequences])
X_influenced = X_influenced.reshape(-1, 14, 782)


# Split X_main and y_data
X_train_main, X_test_main, y_train, y_test = train_test_split(
    X_main, y_data, test_size=0.2, random_state=16
)

# Split X_influenced using the same indices to maintain alignment
X_train_influenced, X_test_influenced = train_test_split(
    X_influenced, test_size=0.2, random_state=16
)

# Fit the model
model.fit(
    {'main_input': X_train_main, 'influenced_input': X_train_influenced},
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=(
        {'main_input': X_test_main, 'influenced_input': X_test_influenced},
        y_test
    )
)

In [32]:
from tensorflow.keras.models import Model

# Test the main_input path
test_main_input = X_train_main[:1]
intermediate_main_output = Model(inputs=main_input, outputs=x).predict(test_main_input)
print("Intermediate main_input output shape:", intermediate_main_output.shape)

# Test the influenced_input path
test_influenced_input = X_train_influenced[:1]
intermediate_influenced_output = Model(inputs=influenced_input, outputs=y).predict(test_influenced_input)
print("Intermediate influenced_input output shape:", intermediate_influenced_output.shape)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 987ms/step
Intermediate main_input output shape: (1, 64)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Intermediate influenced_input output shape: (1, 896)


In [25]:
print("X_main shape:", X_main.shape)
print("X_influenced shape:", X_influenced.shape)
print("y_data shape:", y_data.shape)


X_main shape: (4104, 60, 1)
X_influenced shape: (4104, 14, 782)
y_data shape: (4104, 30)


In [ ]:
# Make predictions
predictions = model.predict({'main_input': X_test_main, 'influenced_input': X_test_influenced})

# Inverse transform the predictions to the original scale
predictions_original_scale = scaler_stock_prices.inverse_transform(predictions)

# Inverse transform the labels to the original scale for comparison
y_test_original_scale = scaler_stock_prices.inverse_transform(y_test)

In [ ]:
def prepare_data( stock_prices):
    sequences = []
    labels = []
